In [1]:
import torch
dataset_type = "CIFAR10"
sample_2D = False
sample_208D = True
save_origin_pic = False

device = torch.device("cuda:0")

#### 是否保存原始图片

In [2]:
import torchvision.transforms as transforms
import torchvision.utils as utils
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

if save_origin_pic == True:
    number = 50000
    transform = transforms.Compose([transforms.ToTensor()])
    train_dataloader = DataLoader(datasets.CIFAR10('./static/data/CIFAR10/CIFAR10', train=True, download=True, transform=transform), batch_size=1, shuffle=False)
    save_path = './static/data/CIFAR10/pic/origin_50k_png/'

    index = 0
    for data in train_dataloader:
        img, labels = data
        utils.save_image(img, save_path + 'pic_' + str(index) + '.png', nrow=1)
        index += 1
        if index >= number:
            break

#### 是否从原始空间中采样

In [3]:
# 保存tensor，不保存图片
# import sys
# python_files_dir = "./python_files/" # python工具包位置
# sys.path.append(python_files_dir)
# import my_tools
# import fid_score as official_fid

# model_files_dir = "./model_files/" # 模型位置
# sys.path.append(model_files_dir)
# import model_files as all_model

# from CIFAR10.models import ResNet20 as classify_model
# from CIFAR10.models import Rob_predictor as my_Rob_predictor
# from CIFAR10.models import BigGAN

# from tqdm import tqdm
# import numpy as np
# import random
# import torchvision.utils as utils

# if sample_208D:
#     # 选择生成模型
#     checkpoints_path = "./model_files/CIFAR10/checkpoints/BigGAN/model=G-best-weights-step=132000.pth"
#     G = BigGAN.Generator().to(device)
#     G.load_state_dict(torch.load(checkpoints_path, map_location=device)["state_dict"])
#     G.eval()
#     number = 50000

    
#     for num in tqdm(range(number)):
#         z = torch.tensor(np.random.RandomState(num).randn(1, 80)).to(torch.float32).to(device)    # latent code
#         label = torch.tensor(random.randrange(10)).unsqueeze(0).to(device)
#         shared_label = G.shared(label)
#         z_and_shared_label = torch.cat((z, shared_label), dim = 1)
#         if num == 0:
#             z_and_shared_labels = z_and_shared_label
#         else:
#             z_and_shared_labels = torch.cat((z_and_shared_labels, z_and_shared_label))
#         img = G(z = z_and_shared_label)                           # NCHW, float32, dynamic range [-1, +1]
#         torch.save(img, "./static/data/CIFAR10/pic/random_50k_tensor/pic_" + str(num) + '.pyt')

#     torch.save(z_and_shared_labels, './static/data/CIFAR10/latent_z/BigGAN_random_50k_tensor_208z_' + str(number) + '.pt')

In [2]:
import sys
python_files_dir = "./python_files/" # python工具包位置
sys.path.append(python_files_dir)
import fid_score as official_fid

model_files_dir = "./model_files/" # 模型位置
sys.path.append(model_files_dir)
import model_files as all_model

from CIFAR10.models import ResNet20 as classify_model
from CIFAR10.models import Rob_predictor as my_Rob_predictor
from CIFAR10.models import BigGAN

from tqdm import tqdm
import numpy as np
import random
import torchvision.utils as utils

if sample_208D:
    # 选择生成模型
    checkpoints_path = "./model_files/CIFAR10/checkpoints/BigGAN/model=G-best-weights-step=392000.pth"
    G = BigGAN.Generator().to(device)
    G.load_state_dict(torch.load(checkpoints_path, map_location=device)["state_dict"])
    G.eval()
    number = 50000

    save_pic_path = "./static/data/CIFAR10/pic/random_50k_png"
    
    for num in tqdm(range(number)):
        z = torch.tensor(np.random.RandomState(num).randn(1, 80)).to(torch.float32).to(device)    # latent code
        label = torch.tensor(random.randrange(10)).unsqueeze(0).to(device)
        shared_label = G.shared(label)
        z_and_shared_label = torch.cat((z, shared_label), dim = 1)
        if num == 0:
            z_and_shared_labels = z_and_shared_label
            labels = label
        else:
            z_and_shared_labels = torch.cat((z_and_shared_labels, z_and_shared_label))
            labels = torch.cat((labels, label), dim=0)
        img = G(z = z_and_shared_label)                           # NCHW, float32, dynamic range [-1, +1]
        img = ((img + 1)/2).clamp(0.0, 1.0) # 变换到[0,1]范围内
        # print(z.shape) # [1, 80]
        # print(shared_label.shape) # [1, 128]
        # print(z_and_shared_label.shape) # [1, 208]
        utils.save_image(img.detach().cpu(), save_pic_path + '/pic_' + str(num) + '.png')
    import datetime
    date_time = datetime.date.today()
    torch.save(z_and_shared_labels, f'./static/data/CIFAR10/latent_z/BigGAN_random_png_208z_{number}_{date_time}.pt')
    torch.save(labels, f'./static/data/CIFAR10/labels/BigGAN_random_png_208z_{number}_{date_time}_labels.pt')

100%|██████████| 50000/50000 [06:47<00:00, 122.59it/s]


#### 是否从2D空间中采样

In [5]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.utils as utils
import numpy as np
from scipy.stats import norm
import torch

# 插值函数
def get_zs_prevent_stick(coordinates, kdTree_2D, latent_z, k=10):
    '''
    nearest_distance: n*k维
    nearest_index: n*k维
    dict_zs: 键是文件的id号, 值是对应的z（后面直接改成数组了，问题不大）
    '''
    print("进入了防止粘在一块~~~~~~~~~~~~~~~~~~")
    # 直接一次查询所有坐标的k个近邻
    nearest_distance, nearest_index = kdTree_2D.query(coordinates, k=k)
    origin_coordinates = kdTree_2D.data # 获取kdtree中原始的坐标
    for i, pos in enumerate(coordinates): #对每一个坐标进行插值
        # pos坐标对应的近邻下标
        pos_nearst_index = nearest_index[i] # 其中有k个index，每个index对应kdTree_2D中的一个2维坐标
        pos_nearst_distance = nearest_distance[i]

        # 最近邻的坐标点，以及最近的距离
        most_nearst_pos = origin_coordinates[pos_nearst_index[0]]
        most_nearst_dis = pos_nearst_distance[0]

        # 利用三角形,找到第二个插值基点，让两边之和越接近第三边，就越是钝角，就越合理
        s1 = most_nearst_dis
        best_index = 1 # 默认第二个最近邻最好
        min_dif = 100
        for j in range(1, k):
            cur_pos = origin_coordinates[pos_nearst_index[j]]
            s2 = pos_nearst_distance[j]
            s3 = np.linalg.norm(most_nearst_pos-cur_pos)
            if (s1 + s2) - s3 < min_dif: # 两边之和大于等于第三边，所以不用绝对值
                min_dif = (s1 + s2) - s3
                best_index = j
        
        temp_z_0 = latent_z[pos_nearst_index[0]].clone().detach()
        temp_z_1 = latent_z[pos_nearst_index[best_index]].clone().detach()
        sum_distance = most_nearst_dis + pos_nearst_distance[best_index]
        z_new = (sum_distance-most_nearst_dis)/(sum_distance) * temp_z_0 + (sum_distance-pos_nearst_distance[best_index])/(sum_distance) * temp_z_1

        z_new = z_new.unsqueeze(0)
        if i == 0:
            zs = z_new
        else:
            zs = torch.cat((zs, z_new), dim=0)

    return zs


# 最开始的版本，就取近邻的向量进行插值，但是不对类的向量进行插值
def get_zs_new_not_class(coordinates, kdTree_2D, latent_z, k=20):
    '''
    coordinates: n个要插值的坐标
    kdTree_2D: 降维后的2D坐标
    latent_z: 生成模型的输入潜向量，和kdTree_2D是一一对应关系
    k: 近邻的数量
    '''
    print("最开始的版本，优化了一下代码~~~~~~~~~~~ k:",k)
    # 直接一次查询所有坐标的k个近邻
    nearest_distance, nearest_index = kdTree_2D.query(coordinates, k=k) #这里的k为固定值
    origin_coordinates = kdTree_2D.data # 获取kdtree中原始的坐标
    for iter in range(len(coordinates)):
        iter_distance = nearest_distance[iter]
        iter_index = nearest_index[iter]
        
        sum_distanceForIter = np.sum(iter_distance) #这k个近邻的距离总和
        for i, index in enumerate(iter_index):
            temp_z = torch.tensor(latent_z[index])
            temp_distance = iter_distance[i]
            w = (sum_distanceForIter-temp_distance)/((k-1)*sum_distanceForIter) #对z进行权重
            if i == 0:
                z = temp_z*w
            else:
                z += temp_z*w

        z[-128:] = latent_z[iter_index[0]][-128:]   #类向量为最近的那一个点的向量

        z = z.unsqueeze(0) # shape:[1*latent_dim]
        if iter == 0:
            zs = z
        else:
            zs = torch.cat((zs, z), dim=0)
        # print(zs.shape)
    return zs


# 用来处理zs的类，方便使用batchsize
class Mydata_sets(Dataset):
    
    def __init__(self, zs):
        super(Mydata_sets, self).__init__()
        self.zs = zs

    def __getitem__(self, index):
        z = self.zs[index]
        return z

    def __len__(self):
        return len(self.zs)

if sample_2D:
    
    # 读取二维坐标数据
    kdTree = torch.load("./static/data/CIFAR10/2D_kdTree/2D_kdTree_50000_png.pt")
    coords = kdTree.data

    # 计算坐标点的协方差矩阵
    cov = np.cov(coords.T)

    # 生成符合多元正态分布的新坐标点
    new_coords = np.random.multivariate_normal(np.mean(coords, axis=0), cov, size=50000)

    latent_z_path="./static/data/CIFAR10/latent_z/BigGAN_random_50k_png_208z_50000.pt"
    latent_z = torch.load(latent_z_path, map_location="cpu") #因为我之前保存数据到了GPU上，所以要回到cpu上才不会出错    

    # from scipy import spatial
    # norm_tree =  spatial.KDTree(data=new_coords)

    zs = get_zs_prevent_stick(new_coords, kdTree, latent_z)

    zs_datasets = Mydata_sets(zs)
    zs_loader = DataLoader(zs_datasets, batch_size=200, shuffle=False, num_workers=1)

    model_files_dir = "./model_files/" # 模型位置
    sys.path.append(model_files_dir)
    import model_files as model_all
    checkpoints_path = "./model_files/CIFAR10/checkpoints/BigGAN/model=G-best-weights-step=392000.pth"
    G = model_all.get_generative_model("CIFAR10").to(device)
    G.load_state_dict(torch.load(checkpoints_path, map_location=device)["state_dict"])
    G.eval()

    first = 0 # 判断是否第一次进入循环
    count = 0
    with torch.no_grad(): # 取消梯度计算，加快运行速度
        for batch_z in zs_loader: 
            z = torch.tensor(batch_z).to(torch.float32).to(device)    # latent code
            imgs = G(z)   
            for i, img in enumerate(imgs):
                img = ((img + 1)/2).clamp(0.0, 1.0) # 变换到[0,1]范围内
                utils.save_image(img.detach().cpu(), f'./static/data/CIFAR10/pic/2D_50k_png/pic_{count}.png')
                count += 1

#### 自定义dataset

In [6]:
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from PIL import Image 

class Mydata_sets(Dataset):
    
    def __init__(self, path, transform=None):
        super(Mydata_sets, self).__init__()
        self.root_dir = path
        self.img_names = os.listdir(self.root_dir)
        self.transform = transform

    def __getitem__(self, index):
        img_name = self.img_names[index]
        img = Image.open(os.path.join(self.root_dir, img_name))
        id_name = torch.tensor(int(img_name[4:-4])) #pic_xx.jpg
        if self.transform is not None:
            img = self.transform(img)
        return img, id_name

    def __len__(self):
        return len(self.img_names)

# # 直接读取tensor数据
# class Mydata_sets2(Dataset):
    
#     def __init__(self, path, transform):
#         super(Mydata_sets2, self).__init__()
#         self.root_dir = path
#         self.img_names = os.listdir(self.root_dir)

#     def __getitem__(self, index):
#         img_name = self.img_names[index]
#         img = torch.load(os.path.join(self.root_dir, img_name), map_location=device)
#         id_name = torch.tensor(int(img_name[4:-4])) #pic_xx.jpg

#         return img, id_name

#     def __len__(self):
#         return len(self.img_names)

if dataset_type == "CIFAR10":
    # transform = transforms.Compose([
    #     transforms.Resize((299, 299), interpolation=Image.BICUBIC),
    #     transforms.ToTensor(),
    #     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    # ])
    # transform = transforms.Compose([
    #     transforms.Resize((299, 299)),
    #     transforms.ToTensor(),
    #     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    # ])
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])
    # transform = transforms.Compose([
    #     transforms.ToTensor(),
    #     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    # ])

#### 开始计算is值

In [7]:
# # 使用vgg_19获取
# python_files_dir = "./python_files/" # python工具包位置
# sys.path.append(python_files_dir)

# model_files_dir = "./model_files/" # 模型位置
# sys.path.append(model_files_dir)
# import model_files as model_all

# inception_model = model_all.get_DNN_model(dataset_type, "VGG19_BN")
# inception_model.load_state_dict(torch.load("./model_files/" + dataset_type + "/checkpoints/classify_model/VGG19_BN.pt", map_location=device))
# inception_model.eval()
# inception_model.to(device)

# # 生成的图像数据集
# generated_dataset = Mydata_sets(path="./static/data/CIFAR10/pic/random_50k/",transform=transform)
# generated_dataloader = DataLoader(generated_dataset, batch_size=32, shuffle=True)

# # 计算预测分数
# predictions = []
# for images, ids in generated_dataloader:
#     images = images.to(device)
#     with torch.no_grad():
#         preds = inception_model(images)
#     predictions.append(torch.softmax(preds, dim=1))

In [8]:
# 使用inception_v3获取
from torchvision.models import inception_v3
from torch.utils.data import DataLoader
import torch.nn.functional as F

# 加载预训练的Inception模型
inception_model = inception_v3(pretrained=True, transform_input=False).eval()
device = torch.device("cuda:1")
inception_model.to(device)


# 生成的图像数据集（假设你有一个名为generated_data的数据集）
generated_dataset = Mydata_sets(path="./static/data/CIFAR10/pic/random_50k_png/",transform=transform)
generated_dataloader = DataLoader(generated_dataset, batch_size=32, shuffle=True)

# 计算预测分数
predictions = []
for images, ids in generated_dataloader:
    images = images.to(device)
    with torch.no_grad():
        images = F.interpolate(images, size=(299, 299), mode='bilinear', align_corners=False)
        preds = inception_model(images)
    predictions.append(torch.softmax(preds, dim=1))

In [9]:
# 使用inception_v3获取cifar10原始数据集的
# from torchvision.models import inception_v3
# import torchvision.datasets as datasets
# from torch.utils.data import DataLoader
# import torch.nn.functional as F

# # 加载预训练的Inception模型
# inception_model = inception_v3(pretrained=True, transform_input=False).eval()
# device = torch.device("cuda:1")
# inception_model.to(device)


# # 生成的图像数据集（假设你有一个名为generated_data的数据集）
# train_dataset = datasets.CIFAR10(root = './static/data/CIFAR10/CIFAR10', train= True, transform=transform)
# train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# # 计算预测分数
# predictions = []
# for images, ids in train_dataloader:
#     images = images.to(device)
#     with torch.no_grad():
#         images = F.interpolate(images, size=(299, 299), mode='bilinear', align_corners=False)
#         preds = inception_model(images)
#     predictions.append(torch.softmax(preds, dim=1))

In [10]:
predictions = torch.cat(predictions, dim=0)
print("predictions.shape: ", predictions.shape)

predictions.shape:  torch.Size([50000, 1000])


In [11]:
fake_probs = predictions
# fake_probs.mean(dim=0, keepdim=True).shape
# 计算Inception Score
kl_divergence = (fake_probs * (fake_probs / fake_probs.mean(dim=0, keepdim=True)).log()).sum(dim=1)
print(kl_divergence.shape)
inception_score = torch.exp(kl_divergence.mean()).item()
print("Inception Score:", inception_score)

torch.Size([50000])
Inception Score: 8.732325553894043


#### 计算fid值

In [12]:
# fid计算模型
dims = 2048
batch_size = 1
num_avail_cpus = len(os.sched_getaffinity(0))
num_workers = min(num_avail_cpus, 8)
block_idx = official_fid.InceptionV3.BLOCK_INDEX_BY_DIM[2048]
fid_model = official_fid.InceptionV3([block_idx], normalize_input=False).to(device)
# fid_model = official_fid.InceptionV3([block_idx]).to(device)
print('fid_model load success!')

if dataset_type == "CIFAR10":
    pic_path_fid1 = './static/data/CIFAR10/pic/origin_50k_png'
    pic_path_fid2 = './static/data/CIFAR10/pic/2D_50k_png'

    
batch_size = 100
m1, s1 = official_fid.compute_statistics_of_path(pic_path_fid1, fid_model, batch_size,
                                    dims, device, num_workers)
m2, s2 = official_fid.compute_statistics_of_path(pic_path_fid2, fid_model, batch_size,
                                    dims, device, num_workers)
fid_value=official_fid.calculate_frechet_distance(m1,s1,m2,s2) 
print(fid_value)

fid_model load success!


100%|██████████| 500/500 [00:50<00:00,  9.90it/s]


sigma1.shape:  (2048, 2048)
sigma2.shape:  (2048, 2048)
14.653057963345361
